In [ ]:
# schiller test --> iodine that stains different on abnormal cells on cervix
# hinsellman --> coloscopy using scope on cervix
# cytology --> pap smear
# biopsy --> biopsy 
# These indicate the RESULTS of these tests, assuming they were carried out. 1 means suspect screening, 0 means okay screening. 

# dx columns mean previous cervical diagnosis 
# stds_number is the sum over all std columns
# Cant figure out what stds_n_diagnosis is, though. 

In [ ]:
import gc 
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_validate, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, RocCurveDisplay, PrecisionRecallDisplay, fbeta_score, make_scorer
from sklearn.impute import MissingIndicator, SimpleImputer
import matplotlib.pyplot as plt
import miceforest as mf
from miceforest import mean_match_default
import seaborn as sns
from lightgbm import LGBMClassifier
import inspect 
import warnings
import scipy.stats as stats
from tempfile import mkdtemp
from joblib import Memory
from shutil import rmtree
from mice_imputer import *
from missing_transformer import *
import prince as pr
import pickle 

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv")

# Encode missings

In [ ]:
df = df.replace({"?": pd.NA})

# Rename columns to be more manageable

In [ ]:
df.columns.values

In [ ]:
new_names = df.columns 
to_rep = {
    "Number" : "n",
    "Contraceptives" : "bc", 
    "Num" : "n",
    "-" : "_",
    "of" : "",
    " " : "_", 
    "(" : "",
    ")" : "",
    "/" : "_",
    ":" : "_", 
    "__" : "_"
}

for key, value in to_rep.items(): 
    new_names = new_names.str.replace(key, value, regex = True)

new_names = new_names.str.lower()

df = df.set_axis(new_names, axis = 1)

df.columns.values

In [ ]:
df = df.apply(pd.to_numeric, axis = 1).convert_dtypes() # convert_dtypes not working without the apply() call. Probably due to the earlier replace statement, but fiddled for an hour and no dice.

# Data Exploration

## Basic variable description

In [ ]:
df.describe().T

In [ ]:
df.sum(axis = 0)

## Iud/smoking/HBC years are always >0 if you have IUD/smoke/HBC.

In [ ]:
def check_zerotrunc(df, bin_col, yr_col):
    return np.any((df[bin_col] == 1) & (df[yr_col] == 0))

In [ ]:
bin_cols = ["iud", "smokes", "hormonal_bc"]
yr_cols = ["iud_years", "smokes_years", "hormonal_bc_years"]

for bin, yr in zip(bin_cols, yr_cols):
    print(f"{bin} == 1 when {yr} == 0?", check_zerotrunc(df, bin, yr))

## Years variables are not strictly integers

In [ ]:
df_mod = df % 1 == 0
df_mod.all(axis = 0)

## Verify that the count of stds is linear combination of all std columns. 

In [ ]:
all((df[df.columns[df.columns.str.startswith("stds_")]].drop(["stds_time_since_first_diagnosis", "stds_time_since_last_diagnosis", "stds_n_diagnosis", "stds_number"], axis = 1).sum(axis = 1) == df.stds_number).dropna())

## Drop time since std diagnoses

In [ ]:
df.drop(df.columns.values[df.columns.str.startswith("stds_time")], axis = 1, inplace = True)

## Check for/drop constant/near-constant columns

In [ ]:
const = df.nunique() == 1
if any(const):
    print("Deleting constant columns: {}".format(df.columns.values[const]))
    df.drop(df.columns.values[const], axis = 1, inplace = True)

# near_const = df.sum(axis = 0) <= 18
# if any(near_const): 
#     print(f"\nDeleting near-constant columns: {df.columns.values[near_const]}")
#     df.drop(df.columns.values[near_const], axis = 1, inplace = True)


## Drop n_diagnosis col

In [ ]:
df.drop(["stds_n_diagnosis"], axis = 1, inplace = True)

## Correlations

In [ ]:
corr = df.corr()
cols = corr.columns

sns.heatmap(
    corr, 
    xticklabels = cols,
    yticklabels = cols
)

In [ ]:
to_hist = ["age", "n_sexual_partners", "first_sexual_intercourse", "n_pregnancies", "smokes_packs_year", "hormonal_bc_years"]
fig, axs = plt.subplots(nrows = 2, ncols = 3)
plt.rcParams['figure.figsize'] = [15, 7]
for ax, var in zip(axs.ravel(), to_hist): 
    df[var].plot.hist(bins = 15, ax = ax, title = var)

# Modeling

In [ ]:
x = df.drop(["smokes", "hormonal_bc", "iud", "stds", "schiller", "biopsy", "hinselmann"], axis = 1)
x["n_stds"] = x["stds_number"]
x.drop("stds_number", axis = 1, inplace = True)
y = df[["biopsy"]].astype("int64")


## Have to have cat/float/int dtypes for lgbm

In [ ]:
x[x.select_dtypes(include=['Int64', 'Float64']).columns.values] = x.select_dtypes(include=['Int64', 'Float64']).astype('float')
x.dtypes

## Instantiate base classifier

In [ ]:
clf = LGBMClassifier(objective = "binary", is_unbalance = True)

## Set some useful fixed parameters for each variable

In [ ]:
template = {
    "objective" : "regression"
}

varparms = {}

keys = x.columns.values[x.isna().any()] 

for i in keys: 

    varparms[i] = template.copy()

    if "stds_" in i: 
        varparms[i]["objective"] = "binary"
        varparms[i]["is_unbalance"] = True

    if (x[i].nunique() > 2) & (np.all(x[i].dropna() % 1 == 0)):
        varparms[i]["objective"] = "poisson"


In [ ]:
varparms

## Set predictive mean matching scheme for imputer

In [ ]:
mean_match = mean_match_default.copy()
mean_match.set_mean_match_candidates(5)

## Setup pipeline components, instantiate pipe

* Impute missing values, tuning either constant imputation (`simple_union`) or mice imputation (`mice_union`)
* Add a single missingness variable for the sexual history related variables, as this constitutes the bulk of the missingness.
* Scale everything
* Apply PCA to std columns 
* Train/tune LGBM classifier

In [ ]:
simple_union = FeatureUnion(
    transformer_list=[
         ('features', SimpleImputer(strategy='median')),
         ('indicator', missing_transformer())
         ]
)

mice_union = FeatureUnion(
    transformer_list=[
         ('features', mice_imputer(mean_match_scheme = mean_match, mice_iterations = 15, variable_parameters = varparms)),
         ('indicator', missing_transformer())]
)

std_cols = np.where(x.columns.str.startswith("stds"))[0]
print(std_cols)

pca_stds = ColumnTransformer(
    [("pca", PCA(n_components = 5),  std_cols)],
    remainder = "passthrough"
)

scaler = StandardScaler()


In [ ]:
#cachedir = mkdtemp()
#memory = Memory(location=cachedir, verbose=0)
pipe = Pipeline(
    #memory = memory,
    steps = [
        ("imputer", simple_union),
        ("scaler", scaler),
        ("pca", pca_stds),
        ("classifier", clf)
    ]
)

## (Some) Marginal distributions for random search

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]

In [ ]:
n = 3
p = .0075
xl = np.floor(np.linspace(0, 1250, 1250))
fig, ax = plt.subplots()

nb = ax.bar(
    xl, 
    stats.nbinom.pmf(xl, n = n, p = p, loc = 1)
)
ax.set_title(f"N Estimators Distribution [nbinom(p = {p}, n = {n})]")
ax.set_ylabel("Probability Mass")
ax.set_xlabel("N Estimators")
plt.show()

In [ ]:
mn = n*(1-p)/p
st = np.sqrt(mn*p**-1)
ps = [.01, .1, .25, .5, .75, .9, .99]
qs = stats.nbinom.ppf(ps, n = n, p = p)
qs = {str(p) : q for p,q in zip(ps, qs)}
qs["mean"] = mn 
qs["std"] = st 
pd.DataFrame(qs, index = ["value"])


In [ ]:
scale = .3
xl = np.linspace(0, 1, 1000)
fig, ax = plt.subplots()

ex = ax.plot(
    xl, 
    stats.expon.pdf(xl, scale = scale)
)
ax.set_title(f"Learning Rate Distribution [expon(scale = {scale})]")
ax.set_ylabel("Probability Density")
ax.set_xlabel("Learning Rate")
plt.show()

In [ ]:
mn = scale
st = scale
ps = [.01, .1, .25, .5, .75, .9, .99]
qs = stats.expon.ppf(ps, scale = scale)
qs = {str(p) : q for p,q in zip(ps, qs)}
qs["mean"] = mn 
qs["std"] = st 
pd.DataFrame(qs, index = ["value"])

## Define random search grid

In [ ]:
lr_dist = stats.expon(scale = scale)
ne_dist = stats.nbinom(n = n, p = p, loc = 1)
nl_dist = stats.randint(2, 51)
md_dist = stats.randint(1, 10)
pc_dist = stats.randint(1, 2)
mc_dist = stats.randint(15, 75)

base_grid = {
    "pca__pca__n_components" : pc_dist,
    "classifier__n_estimators" : ne_dist,
    "classifier__num_leaves" : nl_dist,
    "classifier__max_depth" : md_dist,
    "classifier__learning_rate" : lr_dist,
    "classifier__min_child_samples" : mc_dist
}

grid = [
    {
        "imputer" : [simple_union],
        "imputer__features__strategy" : ["mean", "median"],
        **base_grid
    },
    {
        "imputer" : [mice_union],
        "imputer__features__lgb_iterations" : ne_dist,
        "imputer__features__lgb_learning_rate" : lr_dist,
        "imputer__features__lgb_max_depth" : md_dist,
        "imputer__features__lgb_num_leaves" : nl_dist,
        **base_grid
    }
]

## Setup nested CV folds, flush RAM

In [ ]:
inner_cv = StratifiedKFold(n_splits=5, random_state=874841, shuffle = True)
outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=878571)

In [ ]:
gc.collect()

## Fit model, cleanup, and save

In [ ]:
print(x.columns.values)
std_cols

In [ ]:
f3_scorer = make_scorer(fbeta_score, beta = 2)

rcv = RandomizedSearchCV(
    estimator = pipe,
    param_distributions = grid, 
    scoring = f3_scorer,
    refit = True, 
    cv = inner_cv,
    return_train_score = True,
    n_jobs = 1,
    n_iter = 10,
    random_state = 97417
)

nested_scores = cross_validate(
    rcv, 
    X = x, 
    y = y.values.flatten(), 
    cv = outer_cv, 
    return_estimator = True, 
    scoring = ["average_precision", "balanced_accuracy", "f1", "precision", "recall"],
    n_jobs = 20,
    verbose = 999
)


In [ ]:
try:
    rmtree(cachedir)
except:
    print("No cache to remove.") 

gc.collect()

In [ ]:
def save_obj(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [ ]:
save_obj(nested_scores, "/home/john/gdrive/github/cervical_cancer/rcv.pkl")

In [ ]:
#best_models = nested_scores['estimator']
# mn = nested_scores["test_score"].mean()
# st = nested_scores["test_score"].std()
# [mn - 1.96*st, mn + 1.96 * st]
#for i, model in enumerate(best_models):
#     #print(model.best_estimator_)
      #print(model.best_params_)
#     print(model.best_score_)